In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import copy
import math

In [2]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [9]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [3]:
class Layer_Norm(nn.Module):

  def __init__(self, features, eps=1e-6):
    super(Layer_Norm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [4]:
class Sublayer_Connection(nn.Module):
 
  def __init__(self, size, dropout):
    super(Sublayer_Connection, self).__init__()
    self.dropout = nn.Dropout(p=0.1)
    self.norm = Layer_Norm(size)

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))   

In [5]:
class Encoding(nn.Module):

  def __init__(self, layer, N):
    super(Encoding, self).__init__()
    self.layers = clones(layer, N)
    self.norm = Layer_Norm(layer.size)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return self.norm(x)

In [6]:
class Encoding_Layer(nn.Module):

  def __init__(self, size, self_attention, feed_forward, dropout):
    super(Encoding_Layer, self).__init__()
    self.dropout = nn.Dropout(p=0.1)
    self.self_attention = self_attention
    self.feed_forward = feed_forward
    self.sublayers = clones(Sublayer_Connection(size, dropout), 2)
    self.size = size

  def forward(self, x):
    x = self.sublayer[0](x, lambda x: self.self_attention(x, x, x))
    return self.sublayer[1](x, self.feed_forward)

In [7]:
class Decoding(nn.Module):

  def __init__(self, layer, N):
    super(Decoding, self).__init__()
    self.layers = clones(layer, N)
    self.norm = Layer_Norm(layer.size)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return self.norm(x)

In [8]:
class Decoding_Layer(nn.Module):

  def __init__(self, size, self_attention, feed_forward, dropout):
    super(Decoding_Layer, self).__init__()
    self.dropout = nn.Dropout(p=0.1)
    self.self_attention = self_attention
    self.feed_forward = feed_forward
    self.sublayers = clones(Sublayer_Connection(size, dropout), 3)
    self.size = size

  def forward(self, x):
    x = self.sublayer[0](x, lambda x: self.self_attention(x, x, x))
    x = self.sublayer[1](x, lambda x: self.self_attention(x, x, x))
    return self.sublayer[2](x, self.feed_forward)

In [10]:
def Attention(q, k, v):
  d_k = q.size(-1)
  a = torch.matmul(q, torch.transpose(k,1,2))
  a /= math.sqrt(d_k)
  p_attention = a.softmax(dim=-1)
  return torch.matmul(p_attention, v)

In [11]:
a = torch.randn(32, 72, 512)
b = torch.randn(32, 72, 512)
c = torch.randn(32, 72, 512)
d = Attention(a,b,c)
print(d.size())

torch.Size([32, 72, 512])


In [12]:
class multihead(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(multihead, self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, q, k, v):
    liste = [q, k, v]
    nbatches = q.size(0)
    for i in range(len(liste)):
      for lin, x in zip(self.linears, liste[i]):
        liste[i] = lin(x)

    q = liste[0]
    k = liste[1]
    v = liste[2]

    x = Attention(q, k, v) 
    return x

In [13]:
class FFN(nn.Module):
  def __init__(self, d_model, d_ff):
    super(FFN, self).__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [14]:
class embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(embeddings, self).__init__()
    self.layer = nn.Embedding(vocab, d_model)
    self.d_model = d_model

  def forward(self, x):
    return self.layer(x) * math.sqrt(self.d_model)

In [15]:
class positional_encoding(nn.Module):
  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(positional_encoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    self.d_model = d_model
    
    pe = torch.zeros(max_len, d_model)

    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
    )
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [27]:
"""
class Label_Smoothing(nn.Module):
  # Label Smoothing is a regularization technique that introduces noise for the labels. 
  # This accounts for the fact that datasets may have mistakes in them, so maximizing the likelihood of log( p(y | x) ) directly can be harmful.
  # a set y is correct with probability (1 - epsilon)
  # Label Smoothing regularizes a model based on a softmax with k output values by replacing the hard 0 and 1 \n
  # classification targets with targets of epsilon/(k-1) and (1 - epsilon) respectively
 
  def __init__(self, k, padding_index, epsilon):
    super(Label_Smoothing, self).__init__()
    self.epsilon = epsilon
    self.other = 1 - self.epsilon
    self.k = k
    self.criterion = nn.KLDivLoss(reduction = "sum")
    self.padding_index = padding_index

  def forward(self, x, target):
    return self.criterion(x)
"""

In [31]:
def rate(step, model_size, factor, warmup):
    if step == 0:
        step = 1
    return factor * ((model_size ** (-0.5)) * min((step ** (-0.5)), step * (warmup ** -1.5)))

In [ ]:
# We used the Adam optimizer with β1 = 0.9, β2 = 0.98 and epsilon = (10 ** (-9))